In [2]:
import tkinter as tk
from tkinter import *
import tkinter.messagebox
import pyaudio
import wave
import os
import numpy as np
import math
import operator

In [4]:
from nnmnkwii.datasets import PaddedFileSourceDataset
from nnmnkwii.datasets.cmu_arctic import CMUArcticWavFileDataSource
from nnmnkwii.preprocessing.alignment import DTWAligner
from nnmnkwii.preprocessing import trim_zeros_frames, remove_zeros_frames, delta_features
from nnmnkwii.util import apply_each2d_trim
from nnmnkwii.metrics import melcd
from nnmnkwii.baseline.gmm import MLPG

from os.path import basename, splitext
import os
import sys
import time

import numpy as np
from scipy.io import wavfile
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
import pyworld
import pysptk
from pysptk.synthesis import MLSADF, Synthesizer
import librosa
import librosa.display
import IPython
from IPython.display import Audio
import matplotlib.pyplot as plt
import pickle

/home/phuongdong/miniconda3/envs/voice/lib/python3.6/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.preprocessing.data module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/phuongdong/miniconda3/envs/voice/lib/python3.6/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.mixture.gaussian_mixture module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.mixture. Anything that cannot be imported from sklearn.mixture is now part of the private API.
  warnings.warn(message, FutureWarning)


In [5]:
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
FILE_NAME = 'input/voice.wav'

In [6]:
p = pyaudio.PyAudio()
recording = 0
frames = []

In [7]:
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

In [8]:
def start_record():
    recording = 1
    frames.clear()
    label['text'] = 'Đang ghi âm'
    stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)
    while recording == 1:
        data = stream.read(CHUNK)
        frames.append(data)
        window.update()

In [55]:
def stop_record(): 
    recording = 0
    label['text'] = 'Ghi âm xong'
    print('* stop recording')
    stream.stop_stream()
    stream.close()
    wf = wave.open(FILE_NAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

In [51]:
def play_wav(c):
    print(c)
    p_read = pyaudio.PyAudio()
    wf_read = wave.open(c, 'rb')
    stream_read = p_read.open(format =
                            p_read.get_format_from_width(wf_read.getsampwidth()),
                            channels = wf_read.getnchannels(),
                            rate = wf_read.getframerate(),
                            output = True)
    data = wf_read.readframes(CHUNK)
    while data != '':
        # writing to the stream is what *actually* plays the sound.
        stream_read.write(data)
        data = wf_read.readframes(CHUNK)
        window.update()
    stream_read.close()    
    p_read.terminate()

In [11]:
fftlen = pyworld.get_cheaptrick_fft_size(RATE)
alpha = pysptk.util.mcepalpha(RATE)
order = 24
frame_period = 5
hop_length = int(RATE * (frame_period * 0.001))
max_files = 100 # number of utterances to be used.
test_size = 0.03
use_delta = True

In [12]:
if use_delta:
    windows = [
        (0, 0, np.array([1.0])),
        (1, 1, np.array([-0.5, 0.0, 0.5])),
        (1, 1, np.array([1.0, -2.0, 1.0])),
    ]
else:
    windows = [
        (0, 0, np.array([1.0])),
    ]

In [17]:
with open("models/quang.pkl", "rb") as file: 
    gmm=pickle.load(file)

In [18]:
def import_model(c):
    title_model['text'] = c
    with open("models/" + c + ".pkl", "rb") as file: 
        gmm=pickle.load(file)

In [52]:
def test_one_utt(src_path, disable_mlpg=False, diffvc=True):
    # GMM-based parameter generation is provided by the library in `baseline` module
    if disable_mlpg:
        # Force disable MLPG
        paramgen = MLPG(gmm, windows=[(0,0, np.array([1.0]))], diff=diffvc)
    else:
        paramgen = MLPG(gmm, windows=windows, diff=diffvc)

    fs, x = wavfile.read(src_path)
    print(x)
    x = x.astype(np.float64)
    if len(x.shape)==2:
        x=x.sum(axis=1)/2
    f0, timeaxis = pyworld.dio(x, fs, frame_period=frame_period)
    f0 = pyworld.stonemask(x, f0, timeaxis, fs)
    spectrogram = pyworld.cheaptrick(x, f0, timeaxis, fs)
    aperiodicity = pyworld.d4c(x, f0, timeaxis, fs)

    mc = pysptk.sp2mc(spectrogram, order=order, alpha=alpha)
    c0, mc = mc[:, 0], mc[:, 1:]
    if use_delta:
        mc = delta_features(mc, windows)
    mc = paramgen.transform(mc)
    #if disable_mlpg and mc.shape[-1] != static_dim:
    #    mc = mc[:,:static_dim]
    #assert mc.shape[-1] == static_dim
    mc = np.hstack((c0[:, None], mc))
    if diffvc:
        mc[:, 0] = 0 # remove power coefficients
        engine = Synthesizer(MLSADF(order=order, alpha=alpha), hopsize=hop_length)
        b = pysptk.mc2b(mc.astype(np.float64), alpha=alpha)
        waveform = engine.synthesis(x, b)
    else:
        spectrogram = pysptk.mc2sp(
            mc.astype(np.float64), alpha=alpha, fftlen=fftlen)
        waveform = pyworld.synthesize(
            f0, spectrogram, aperiodicity, fs, frame_period)
        
    return waveform

In [58]:
def vc():
    src_path="input/voice.wav"
    tgt_path="output/voice.wav"
    w_MLPG = test_one_utt(src_path, disable_mlpg=False)
    wavfile.write(tgt_path, RATE, w_MLPG)

In [ ]:
window = tk.Tk()
window.geometry('500x300')
window.title('Voice conversion')
model_names = [
    'dong',
    'quang',
    'duc',
    'an',
    'tuyen',
    'thang'
]
button_model = tk.Frame(window, padx=150, pady=100)
button_model.pack(fill=BOTH)
title_model = Menubutton(button_model, width=20, padx=10,
                                pady=5, relief=RAISED)
title_model.grid(row=0, column=0, padx=20, pady=5)
title_model['text'] = 'Chọn giọng chuyển'
title_model.menu = Menu(title_model)
title_model["menu"] = title_model.menu
for c in model_names:
    title_model.menu.add_command(label=c, command=lambda c=c: import_model(c))
button_start = tk.Button(window, text='Ghi âm', width=15, command=start_record)
button_start.place(x=70, y=50)
button_stop = tk.Button(window, text='Dừng ghi âm', width=15, command=stop_record)
button_stop.place(x=190, y=50)
button_play = tk.Button(window, text='Phát lại', width=15, command=lambda c='input/voice.wav': play_wav(c))
button_play.place(x=310, y=50)
button_vc = tk.Button(window, text='Chuyển giọng', width=15, command=vc)
button_vc.place(x=130, y=150)
button_play_2 = tk.Button(window, text='Kiểm tra', width=15, command=lambda c='output/voice.wav': play_wav(c))
button_play_2.place(x=250, y=150)
label = tk.Label(window)
label.place(x=220, y=200)
tk.mainloop()

* stop recording
[b'\x92\n\x8f\n\xb8\nU\n\x1d\n\x88\t\xd5\t\xe9\t\x96\t\xd2\t\xa3\t\xb1\t\xcb\t\x0b\tK\x08\t\x08A\x08\xae\x07\xda\x07c\x08\x15\x08\x93\x07\xa5\x07=\x08L\x08\xe9\x07\'\x08\x14\x08\xca\x078\x08\x05\x08\xa5\x07d\x07Q\x07\xf5\x07\x95\x07\x0f\x08R\x07\x19\x07V\x07\x17\x07\xa2\x073\x08/\x08\x8e\x07N\x07k\x07\xd8\x07\xf4\x07u\x07\xa2\x06l\x07G\x07T\x07@\x08G\x08j\x08\xe5\x08\xfa\x07\x9f\x08\x0e\x08\x7f\x07w\x07[\x07\x85\x07\xab\x074\x07\x93\x06R\x07\xc3\x06\xb2\x06P\x07r\x06\xfe\x056\x06#\x06\xf0\x05M\x06\xf8\x05I\x06\x91\x052\x06N\x06\x82\x05U\x05m\x05\x0e\x06\xbf\x05\xf4\x05\x01\x06r\x05,\x067\x06.\x06\x8e\x06\xab\x06\xa4\x06\xb0\x06\xd4\x05\x8c\x05b\x05\xe6\x04:\x04D\x04\x9b\x04<\x04\xaa\x04r\x04\xa8\x04c\x04\xc4\x03\x11\x04\xa7\x03`\x03\x11\x04\x96\x03\xb5\x03\x06\x04\x8b\x04\x87\x03f\x03j\x03I\x02\xd6\x02w\x03\xde\x02\xc7\x02\x06\x03\x96\x02\xe1\x02\xce\x02<\x02\x18\x02\x01\x02\xee\x01e\x02\xae\x02]\x02r\x02\x07\x02\xb1\x02\x18\x03\x9c\x02G\x02%\x02\xb8\x01A\x03\xb1\x03\x

Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/phuongdong/miniconda3/envs/voice/lib/python3.6/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-59-a8ec0f72c83e>", line 30, in <lambda>
    button_play_2 = tk.Button(window, text='Kiểm tra', width=15, command=lambda c='output/voice.wav': play_wav(c))
  File "<ipython-input-51-09022e9ea70a>", line 4, in play_wav
    wf_read = wave.open(c, 'rb')
  File "/home/phuongdong/miniconda3/envs/voice/lib/python3.6/wave.py", line 499, in open
    return Wave_read(f)
  File "/home/phuongdong/miniconda3/envs/voice/lib/python3.6/wave.py", line 163, in __init__
    self.initfp(f)
  File "/home/phuongdong/miniconda3/envs/voice/lib/python3.6/wave.py", line 143, in initfp
    self._read_fmt_chunk(chunk)
  File "/home/phuongdong/miniconda3/envs/voice/lib/python3.6/wave.py", line 260, in _read_fmt_chunk
    raise Error('unknown format: %r' % (wFormatTag,))
wave.Error: unknown format

output/voice.wav


Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/phuongdong/miniconda3/envs/voice/lib/python3.6/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-59-a8ec0f72c83e>", line 30, in <lambda>
    button_play_2 = tk.Button(window, text='Kiểm tra', width=15, command=lambda c='output/voice.wav': play_wav(c))
  File "<ipython-input-51-09022e9ea70a>", line 4, in play_wav
    wf_read = wave.open(c, 'rb')
  File "/home/phuongdong/miniconda3/envs/voice/lib/python3.6/wave.py", line 499, in open
    return Wave_read(f)
  File "/home/phuongdong/miniconda3/envs/voice/lib/python3.6/wave.py", line 163, in __init__
    self.initfp(f)
  File "/home/phuongdong/miniconda3/envs/voice/lib/python3.6/wave.py", line 143, in initfp
    self._read_fmt_chunk(chunk)
  File "/home/phuongdong/miniconda3/envs/voice/lib/python3.6/wave.py", line 260, in _read_fmt_chunk
    raise Error('unknown format: %r' % (wFormatTag,))
wave.Error: unknown format

output/voice.wav


Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/phuongdong/miniconda3/envs/voice/lib/python3.6/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-59-a8ec0f72c83e>", line 30, in <lambda>
    button_play_2 = tk.Button(window, text='Kiểm tra', width=15, command=lambda c='output/voice.wav': play_wav(c))
  File "<ipython-input-51-09022e9ea70a>", line 4, in play_wav
    wf_read = wave.open(c, 'rb')
  File "/home/phuongdong/miniconda3/envs/voice/lib/python3.6/wave.py", line 499, in open
    return Wave_read(f)
  File "/home/phuongdong/miniconda3/envs/voice/lib/python3.6/wave.py", line 163, in __init__
    self.initfp(f)
  File "/home/phuongdong/miniconda3/envs/voice/lib/python3.6/wave.py", line 143, in initfp
    self._read_fmt_chunk(chunk)
  File "/home/phuongdong/miniconda3/envs/voice/lib/python3.6/wave.py", line 260, in _read_fmt_chunk
    raise Error('unknown format: %r' % (wFormatTag,))
wave.Error: unknown format